# Processing CWB Daily Analysis

The Central Weather Bureau (CWB) published a daily weather briefing (天氣概況) on its [official website](https://www.cwb.gov.tw/V8/C/W/index.html) everyday. This short analysis usually contains:

- the major synaptic weather systems around Taiwan,
- the basic weather conditions of Taiwan (cloud, temperature, and precipitation), and
- the prospects of weather conditions of Taiwan.
- Weather warnings are specified if exist.
- The final part of the briefing is about the wind condition on the ocean. 

Here we show two examples:
<blockquote>
(2012)２月２９日８時天氣概況：

　　一、高氣壓１０２６百帕，在北緯３８度，東經１２７度，即在韓國，向東緩慢移動。

　　二、今（２９日）白天起大陸冷氣團減弱，氣溫略回升，臺灣中部以北、東北部、東部、東南部地區及金門、馬祖有短暫雨，南部地區及澎湖為多雲的天氣；明（３月１）日臺灣東北部、東部及東南部地區有局部短暫雨，其他地區及澎湖、金門、馬祖為多雲到晴的天氣。今明兩天北部、中南部及澎湖、金門、馬祖易有局部霧。

　　三、海上強風特報：
１、受東北季風影響，臺灣北部海面、臺灣海峽北部、臺灣東南部海面及臺灣海峽南部平均風力可達６級，最大陣風８級，船隻請注意。今（２９）日至明（３月１）日上述各海面平均風力將持續減弱。
２、受東北季風影響，巴士海峽及廣東海面平均風力可達６級，最大陣風８級，船隻請注意。今（２９日）晚起至明（３月１日）晨間南海平均風力將增強至６級，最大陣風８級，船隻請注意。明（３月１）日廣東海面平均風力將稍減弱。
</blockquote>

Another example:

<blockquote>
(2023)６月２８日８時天氣概況：
    
一、天氣特報：請參照本局發布之最新豪(大)雨特報。
    
二、高氣壓１０１８百帕，在北緯２９度，東經１３９度，即在日本南方海面，中心近似滯留。
    
三、今（２８日）白天南方雲系影響，臺灣東部及東南部地區有短暫陣雨，南部地區及澎湖亦有局部短暫陣雨，其他地區及金門、馬祖為多雲到晴，午後中部以北地區及東北部山區有局部短暫雷陣雨，山區並有局部大雨發生的機率。晚起至明（２９）日臺灣各地及馬祖多雲到晴，東部、東南部地區、恆春半島及澎湖、金門有局部短暫陣雨，午後西半部地區及其他山區有局部短暫雷陣雨，山區並有局部大雨，尤其中南部山區有局部短延時豪雨發生的機率。今晚至明晨基隆北海岸、宜蘭、恆春半島及南部沿海空曠地區易有８至９級強陣風，請注意。
    
四、黃海南部有局部霧，請注意。
    
五、海上強風特報：
    
１、臺灣北部海面平均風力可達６級，最大陣風８級，船隻請注意。今（２８日）下午起臺灣東北部海面平均風力將增強至６到７級，雷雨區最大陣風１０級；晚起臺灣海峽南部平均風力將增強至６級，最大陣風８級，船隻請特別注意。明（２９日）晨起臺灣北部海面平均風力將再增強至６到７級，最大陣風９級；臺灣海峽北部平均風力將再增強至６級，最大陣風８級，船隻請特別注意。明（２９）日臺灣東北部海面平均風力將稍減弱。
    
２、巴士海峽及東沙島海面平均風力可達６級，雷雨區最大陣風９級，船隻請注意。明（２９日）晚起南沙島海面平均風力將增強至６級，雷雨區最大陣風９級，船隻請注意。明（２９）日巴士海峽及東沙島海面平均風力將稍減弱。
    
３、今（２８日）晚起花鳥山海面、浙江海面及東海平均風力將增強至６級，最大陣風８級，船隻請注意。明（２９日）上午起黃海南部平均風力將增強至６級，雷雨區最大陣風９級；晚起花鳥山海面及浙江海面平均風力將增強至６到７級，最大陣風９級，船隻請特別注意。
</blockquote>

We have collected this briefing data in 2012-02-29 ~ 2020-12-31, and we want to use the analysis text to train a language model. Before we do that, we need to process the text data and form a corpus.

In [1]:
import os
import numpy as np
import pandas as pd

DATAROOT = '../../data/cwb_daily/'

# Walk through the sub-directories
file_list = []
for root, dirs, files in os.walk(DATAROOT):
    for name in files:
        if name.endswith('_1100.W01.dat'):
            date = name.replace('_1100.W01.dat','').replace('-','')
            url = os.path.join(root, name)
            file_list.append({'date':date, 'furi':url})

file_list = pd.DataFrame(file_list)
print(file_list.shape)
print(file_list.head())

(3061, 2)
       date                                               furi
0  20120229  ../../data/cwb_daily/2012\2012-02-29_1100.W01.dat
1  20120301  ../../data/cwb_daily/2012\2012-03-01_1100.W01.dat
2  20120302  ../../data/cwb_daily/2012\2012-03-02_1100.W01.dat
3  20120303  ../../data/cwb_daily/2012\2012-03-03_1100.W01.dat
4  20120304  ../../data/cwb_daily/2012\2012-03-04_1100.W01.dat


In [2]:
file_list.to_csv('../../data/flist_dailybriefing.csv', index=False)

In [3]:
furi = file_list['furi'].iloc[0]

with open(furi, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
print(lines)

['07fW01296\n', '中央氣象局氣象報告\n', '１０１年２月２９日１１時０分發佈\n', '２月２９日８時天氣概況：\n', '\n', '\u3000\u3000一、高氣壓１０２６百帕，在北緯３８度，東經１２７度\n', '，即在韓國，向東緩慢移動。\n', '\n', '\u3000\u3000二、今（２９日）白天起大陸冷氣團減弱，氣溫略回升，\n', '臺灣中部以北、東北部、東部、東南部地區及金門、馬祖有短\n', '暫雨，南部地區及澎湖為多雲的天氣；明（３月１）日臺灣東\n', '北部、東部及東南部地區有局部短暫雨，其他地區及澎湖、金\n', '門、馬祖為多雲到晴的天氣。今明兩天北部、中南部及澎湖、\n', '金門、馬祖易有局部霧。\n', '\n', '\u3000\u3000三、海上強風特報：\n', '１、受東北季風影響，臺灣北部海面、臺灣海峽北部、臺灣東\n', '南部海面及臺灣海峽南部平均風力可達６級，最大陣風８級，\n', '船隻請注意。今（２９）日至明（３月１）日上述各海面平均\n', '風力將持續減弱。\n', '２、受東北季風影響，巴士海峽及廣東海面平均風力可達６級\n', '，最大陣風８級，船隻請注意。今（２９日）晚起至明（３月\n', '１日）晨間南海平均風力將增強至６級，最大陣風８級，船隻\n', '請注意。明（３月１）日廣東海面平均風力將稍減弱。\n', '\n', '\n']


In [4]:
furi = file_list['furi'].iloc[3060]

with open(furi, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
print(lines)

['氣象報告\n', '１０９年１２月３１日１１時０分發布\n', '１２月３１日８時天氣概況：\n', '\n', '\u3000\u3000天氣特報：\n', '\u3000\u3000請參照本局發布之最新陸上強風特報、低溫特報。\n', '\n', '\u3000\u3000高氣壓１０４０百帕，在北緯３０度，東經１１６度，即\n', '在安徽，向東移動，時速１０公里。\n', '\n', '\u3000\u3000今、明（３１日、１月１日）兩天寒流影響，各地天氣非\n', '常寒冷；臺灣東半部及大臺北地區有局部短暫雨，其他地區及\n', '澎湖、金門、馬祖為多雲到晴；今日中部以北、東半部（含蘭\n', '嶼、綠島）、恆春半島沿海地區及澎湖、金門、馬祖有長浪發\n', '生的機率，請注意。\n', '\n', '\u3000\u3000海上強風特報：\n', '１、東北風明顯偏強，臺灣海峽平均風力７至８級，最大陣風\n', '１０級；臺灣北部海面、臺灣東北部海面及臺灣東南部海面平\n', '均風力６至７級，最大陣風９級，船隻請特別注意。今（３１\n', '）日臺灣東北部海面及臺灣海峽南部平均風力將稍減弱。明（\n', '１月１）日臺灣東北部海面及臺灣海峽南部、臺灣北部海面、\n', '臺灣東南部海面及臺灣海峽北部平均風力將稍減弱。\n', '２、東北風明顯偏強，巴士海峽及南海平均風力７至８級，最\n', '大陣風１０級；廣東海面平均風力６至７級，最大陣風９級，\n', '船隻請特別注意。明（１月１）日巴士海峽、廣東海面、東沙\n', '島海面及中西沙島海面平均風力將稍減弱。\n', '３、高壓梯度影響，浙江海面平均風力６至７級，最大陣風９\n', '級；黃海南部、花鳥山海面及東海平均風力可達６級，最大陣\n', '風８級，船隻請特別注意。今（３１）日浙江海面平均風力將\n', '稍減弱。明（１月１）日黃海南部、花鳥山海面、浙江海面及\n', '東海平均風力將稍減弱。\n']


We want to keep only the weather-related description as the training corpus, so we need a parser to collect the relevant parts of the text file. From the two examples above, we found a few characteristics:

- The header lines are inconsistent. Hence, we cannot select text by line numbers.
- Paragraphs are broken into multiple lines and they are separated by a blank line.

We can design a parser based on such characteristics.


In [10]:
def parse_cwb_briefing(furi):
    ''' Read and parse the CWB daily briefing text. '''
    # Read data into lines
    with open(furi, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # Detect empty lines as paragraph breaks
    breaks =  [i for i, e in enumerate(lines) if e=='\n']
    # Concatenate paragraphs
    doc = ''
    for i in range(len(breaks)-1):
        tmp = lines[breaks[i]+1:breaks[i+1]]                          # extract paragraph one by one
        tmp = [s.replace('\n','').replace('\u3000','') for s in tmp]  # remove line breaks and multi-byte space
        paragraph = ''.join(tmp)
        if paragraph!='':
            doc = doc + paragraph + '\n'
    # done
    return(doc)

tmp = parse_cwb_briefing(file_list['furi'].iloc[3060])
print(tmp)

天氣特報：請參照本局發布之最新陸上強風特報、低溫特報。
高氣壓１０４０百帕，在北緯３０度，東經１１６度，即在安徽，向東移動，時速１０公里。
今、明（３１日、１月１日）兩天寒流影響，各地天氣非常寒冷；臺灣東半部及大臺北地區有局部短暫雨，其他地區及澎湖、金門、馬祖為多雲到晴；今日中部以北、東半部（含蘭嶼、綠島）、恆春半島沿海地區及澎湖、金門、馬祖有長浪發生的機率，請注意。



In [13]:
## Collect all documents
import json
import pickle

data = []
corpus = ''
for i in range(file_list.shape[0]):
    row = file_list.iloc[i,:]
    doc = parse_cwb_briefing(row['furi'])
    corpus = corpus + doc + '\n'
    data.append({'date':row['date'], 'content':doc})

print(len(data))
print(len(corpus))

3061
1568721


In [14]:
# Output
with open('../../data/cwb_briefing_all.txt', 'w', encoding='utf-8') as f:
    f.write(corpus)

with open('../../data/cwb_briefing_all.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)


Now, we have a single text file as the corpus. We will use this file to train a LLM.

## Dive into the Structure of the Daily Briefing

The length and number of sections of daily briefing content varied from day to day. We can roughly categorize the content into a few sections:

1. **header**: report id, date and time information.
2. **天氣特報**: optional.
3. **synoptic-systems**: the type of system (high, low, or typhoon), its location (lon, lat), and its motion (direction and speed).
4. **briefing**: usually starting with "今（DD）受XX影響，台灣....", describing the general weather conditions.
5. **海上特報**: fog and strong wind.

Generally, the report always has sections 1, 3, 4, and 5. Section 2 only shows up when there are special warnings. Also, section 5 sometimes consists of multiple sub-sections. We want to separate the sections for further analysis.